In [284]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [285]:
df_ = pd.read_csv("https://raw.githubusercontent.com/thistleknot/Python-Stock/master/data/raw/states.csv").set_index('States')

In [286]:
df = sm.add_constant(df_)

In [289]:

#nested functions (kind of like classes, get access to parent functions members).
def filter_df(x,y):
    
    def print_df(aboveMedian,belowMedian):
        
        data = df[[y,*x]].sort_values(kind="quicksort", by=y, ascending=True)
        #abovePos = [i for i, item in enumerate(data.columns) if item in aboveMedian]
        #belowPos = [i for i, item in enumerate(data.columns) if item in belowMedian]

        print(data.describe())
        med = pd.DataFrame(np.median(data,axis=0)).T
        med.columns = data.columns
        
        data = data.iloc[pd.DataFrame(np.array(data[list(aboveMedian)])>=np.array(med[list(aboveMedian)])).replace(False,np.NaN).dropna().index]
        
        data = data.iloc[pd.DataFrame(np.array(data[list(belowMedian)])<=np.array(med[list(belowMedian)])).replace(False,np.NaN).dropna().index]
        
        plt.plot(data[y])
        plt.show()
        data[y].hist()
        plt.show()
        print(data.describe())
        return(data)
    
    aboveMedian = widgets.SelectMultiple(
        options=df[list(x)].columns,
        #value=['Oranges'],
        #rows=10,
        description='AboveMedian',
        disabled=False
    )
    
    belowMedian = widgets.SelectMultiple(
        options=df[list(x)].columns,
        #options=aboveMedian(x),
        #df[list(x)].columns,
        #value=['Oranges'],
        #rows=10,
        description='BelowMedian',
        disabled=False
    )
    
    interact(print_df,aboveMedian=aboveMedian,belowMedian=belowMedian)
    
def derive_xnames(y):
    x_ = widgets.SelectMultiple(
        options=df.columns[~df.columns.isin([y])],
        value=list(df.columns[~df.columns.isin([y])]),
        disabled=False
    )
    
    interact(filter_df,x=x_,y=y)

In [290]:

y=widgets.Select(options=df_.columns,disabled=False)

interact(derive_xnames,y=y)


interactive(children=(Select(description='y', options=('Poverty', 'Infant Mort', 'White', 'Crime', 'Doctors', …

<function __main__.derive_xnames(y)>